In [ ]:
# For the first time running, install following pakages (Remove # below)
 # Pkg.add("VoronoiFVM")
 # Pkg.add("ExtendableGrids")
 # Pkg.add("PyPlot")
 # Pkg.add("Plots")
 # Pkg.add("DataFrames")
 # Pkg.add("CSV")

In [ ]:
# Guide (based on Julia v1.5.1 and VoronoiFVM.jl v0.10.2)
 # Input parameter
 # Run
 # Read Bottom_T_MFD.csv => Find time where sublimation temperature reached (t1)
 # Re-run
 # Read Position_MFD.csv => Find time where length reached (t2)
 # Drying time = t1 + t2

In [ ]:
using VoronoiFVM
using ExtendableGrids
using PyPlot
using Plots

Length=0.042;
edge_n=300; # number of edges
X1=collect(0.0:Length/edge_n:Length)
grid=ExtendableGrids.simplexgrid(X1)
ExtendableGrids.plot(grid,Plotter=PyPlot)

In [ ]:
# Parameter
density1=63;
density2=917;
cp2=1967.8;
pbwi=0.04;
picei=1-pbwi;
pw=0.92;
k2=2.30;
Tsub=256.15; # Sublimation temperature
Hsub=2.840*10^6;
Hw=242345;

In [ ]:
function storage!(f,u,node) # Storage term  # 1:T2, 2:X(t)
        f[1]=density2*cp2*u[1]
        f[2]=(density2-density1)*(Hsub*picei)*u[2]
end

In [ ]:
function flux!(f,u,edge) # Flux term
        f[1]=-k2*(u[1,2]-u[1,1])
        f[2]=0
end

In [ ]:
function reaction!(f,u,node) # Reaction term
        f[1]=-Hw*pbwi
        f[2]=-Hw*Length*pw
end

In [ ]:
physics=VoronoiFVM.Physics(num_species=2,flux=flux!,storage=storage!,reaction=reaction!)

In [ ]:
system=VoronoiFVM.DenseSystem(grid,physics)
enable_species!(system,1,[1]) # System, ispec::Integer, regions::AbstractArray{T,1}
enable_species!(system,2,[1])

In [ ]:
inival=unknowns(system)
solution=unknowns(system)
for i=1:num_nodes(grid)
    inival[1,i]=236.85      # Initial temperature
    inival[2,i]=0           # Initial position
end

In [ ]:
using Plots
using DataFrames
using CSV
df_a=DataFrame()
df_b=DataFrame()
df_c=DataFrame()

let
t=0.0; del_t=10.00
    @gif for i=1:360*3  # Sublimation end
         t=t+del_t
        
        push!(df_a,(A=t/3600,B=solution[1,300]-273.15)) # Interface temperature
        CSV.write("Interface_T_MFD.csv",df_a,writeheader=false)
        
        push!(df_b,(A=t/3600,B=solution[1,1]-273.15)) # Bottom temperature
        CSV.write("Bottom_T_MFD.csv",df_b,writeheader=false)
        
        push!(df_c,(A=t/3600,B=solution[2,1])) # Interface position
        CSV.write("Position_MFD.csv",df_c,writeheader=false)
        
        solve!(solution,inival,system,tstep=del_t)
    
        @views begin
        P0=heatmap(reshape(solution[1,:],length(X1),1),colorbar=:right,color=:viridis,clim=(220,260),yflip=false)
        P1=Plots.plot(solution[1,:],ylim=(220,260),yflip=false)
        P2=Plots.plot(solution[2,:],ylim=(0,0.042),yflip=false)
        P=Plots.plot(P0,P1,P2,layout=(1,3))
        end
        inival.=solution; del_t*=1.00
   end
end